In [1]:
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.engine.topology import Layer
import keras.initializers 
from keras.layers import Lambda, Input 
from keras.models import Model

C:\Users\Ni-Co\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [33]:
constants = {
    'wgs84_earth_mu': K.constant(3.986004418e14, name='mu')
}

keplerian_elements = K.variable([7999e3, 1e-4, 0, 0.2, 0.1, 0.7], name='mean_elements')

def keplerian_orbit(time, keplerian_elements):
    a, ex, ey, hx, hy, m = tf.unstack(keplerian_elements, axis=0)
    mu=constants['wgs84_earth_mu']
    mean_motion = K.sqrt(mu/a)/a
    
    da, dex, dey, dhx, dhy = [K.zeros_like(time) for _ in range(5)]
    dm = mean_motion * time
    orbit_update = tf.stack([da, dex, dey, dhx, dhy, dm], axis=1)
    
    return keplerian_elements + orbit_update

propagate = Lambda(keplerian_orbit, arguments={'keplerian_elements':keplerian_elements}, name='keplerian_propagator')

In [34]:
time = Input(shape=(1,), dtype=np.float32, name='date')
ephemeris = propagate(time)
propagator = Model(time, ephemeris)

In [35]:
propagator.compile(optimizer='sgd', loss='mean_squared_error')
propagator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
date (InputLayer)            (None, 1)                 0         
_________________________________________________________________
keplerian_propagator (Lambda (None, 6, 6)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [36]:
propagator.predict(np.arange(0, 7200, 600))

array([[[7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01],
        [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01],
        [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01],
        [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01],
        [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01],
        [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01]],

       [[7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01],
        [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01],
        [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
         1.0000000e-01, 6.9999999e-01],
        [7.999000

In [52]:
tf.reset_default_graph()

constants = {
    'wgs84_earth_mu': tf.constant(3.986004418e14, name='mu')
}

keplerian_elements = tf.get_variable('mean_elements', initializer=tf.constant([7999e3, 1e-4, 0, 0.2, 0.1, 0.7]))

def keplerian_orbit(keplerian_elements, time):
    a, ex, ey, hx, hy, m = tf.unstack(keplerian_elements, axis=0)
    mu=constants['wgs84_earth_mu']
    mean_motion = tf.sqrt(mu/a)/a
    
    da, dex, dey, dhx, dhy = [tf.zeros_like(time) for _ in range(5)]
    dm = mean_motion * time
    orbit_update = tf.stack([da, dex, dey, dhx, dhy, dm], axis=0)
    
    return keplerian_elements + orbit_update

time = tf.placeholder(tf.float32, shape=(None,1))
ephemeris = keplerian_orbit(keplerian_elements, time)

In [53]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(ephemeris, feed_dict={time:np.arange(0, 7200, 600).reshape(-1, 1)}))

[[[7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]
  [7.9990000e+06 9.9999997e-05 0.0000000e+00 2.0000000e-01 1.0000000e-01
   6.9999999e-01]

(?, ?)


In [21]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [22]:
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

[3. 7.]


In [31]:
class Kepler(Layer):

    def __init__(self, equinoctial_orbit, **kwargs):
        self.orbit = equinoctial_orbit
        self.mu = K.constant(3.986004418e14)
        super(Kepler, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='equinoctial_orbit', 
                                      shape=(1, 6),
                                      initializer=keras.initializers.constant(self.orbit),
                                      trainable=True)
        super(Kepler, self).build(input_shape)  # Be sure to call this at the end

    def call(self, time):
        semi_major_axis = self.orbit[0]
        mean_motion = K.sqrt(self.mu/semi_major_axis)/semi_major_axis
        orbit_update = [0, 0, 0, 0, 0, mean_motion] * time
        return self.orbit + orbit_update

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 6)

In [35]:
time = Input(shape=(1,), dtype=np.float32, name='date')
ephemeris = Kepler([7999e3, 1e-4, 0, 0.2, 0.1, 0.7], name='propagation')(time)
propagator = Model(time, ephemeris)
propagator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
date (InputLayer)            (None, 1)                 0         
_________________________________________________________________
propagation (Kepler)         (None, 6)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [29]:
propagator.predict(np.arange(0, 7200, 600))

array([[7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 6.9999999e-01],
       [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 1.2295008e+00],
       [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 1.7590015e+00],
       [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 2.2885025e+00],
       [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 2.8180032e+00],
       [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 3.3475039e+00],
       [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 3.8770049e+00],
       [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 4.4065056e+00],
       [7.9990000e+06, 9.9999997e-05, 0.0000000e+00, 2.0000000e-01,
        1.0000000e-01, 4.9360061e+00],
       [7.9990000e+06, 9.9999997e-05,